In [1]:
import openai
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
import os
import random

# .env 파일에서 OpenAI API 키 로드
load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [2]:
# 태그 목록
tags = [
    "병원", "아기 용품", "의학 정보", "태몽", "사주", 
    "별자리 운세", "십이지 운세", "부모 고민 상담", 
    "예방 접종", "육아 보조금", "기타"
]

# 조건 설정
ages = ["20대 후반", "30대 초반", "30대 후반"]
experiences = ["초보", "둘째를 키우는"]
genders = ["엄마", "아빠"]
child_ages = ["0세", "1세", "2세"]
seasons = ["봄", "여름", "가을", "겨울"]
special_situations = ["일반", "쌍둥이", "장애아", "다문화 가정"]

# 비율 설정
age_ratio = [1, 1, 1]
experience_ratio = [1, 1]
gender_ratio = [6, 4]  # 설문조사에는 8:2로 응답했으나, 데이터의 정확성 증가를 위해 6:4로 설정
child_age_ratio = [1, 1, 1]
season_ratio = [1, 1, 1, 1]
special_situation_ratio = [7, 1, 1, 1]

def generate_prompt(tag, age, experience, gender, child_age, season, special_situation):
    base_prompt = (
        f"{age} {experience} {gender}가 {child_age} 자녀를 키우고 있습니다. "
        f"현재 계절은 {season}입니다. "
    )
    
    if special_situation != "일반":
        base_prompt += f"이 가정은 {special_situation} 상황입니다. "
    
    base_prompt += (
        f"{child_age} 자녀를 키우는 {gender}의 입장에서, {tag}에 관한 질문을 하나 생성해주세요. "
        f"질문은 30자 내외로 작성하고, 최근에 겪은 특정 상황이나 문제점을 포함하며, "
        f"실제 부모가 사용할 법한 자연스러운 표현으로 작성해주세요. "
    )
    
    if tag == "태몽":
        base_prompt += (
            "태몽은 태아를 잉태하거나 낳게 될 징조의 꿈입니다. "
            "임신 중이거나 임신을 계획 중인 부모의 태몽에 관한 질문을 생성해주세요. "
            "예를 들어, 특정 꿈의 의미나 태몽과 실제 아이의 관계 등에 대해 물어볼 수 있습니다."
        )
    elif tag == "사주":
        base_prompt += (
            "사주는 태어난 순간 우주로부터 부여받은 운명의 암호로, 한 사람의 생애를 담고 있는 코드입니다. "
            "아이의 사주나 부모와 아이의 사주 궁합 등에 관한 질문을 생성해주세요. "
            "예를 들어, 아이의 성격, 재능, 미래 등에 대해 사주를 통해 알고 싶어하는 질문을 만들 수 있습니다."
        )
    elif tag == "별자리 운세":
        base_prompt += (
            "별자리 운세는 생년월일을 통해 역산된 별자리를 바탕으로 운세를 점치는 것입니다. "
            "아이의 별자리나 부모와 아이의 별자리 관계 등에 대한 질문을 생성해주세요. "
            "예를 들어, 특정 별자리의 성격 특성이나 아이의 별자리에 따른 양육 방법 등을 물어볼 수 있습니다."
        )
    elif tag == "십이지 운세":
        base_prompt += (
            "십이지 운세는 12개의 동물 띠로 나누어 운세를 보는 방법입니다. "
            "아이의 띠나 부모와 아이의 띠 궁합 등에 관한 질문을 생성해주세요. "
            "예를 들어, 특정 띠의 운세나 아이의 띠에 따른 성격, 재능 등에 대해 물어볼 수 있습니다."
        )
    
    return base_prompt

# 부적절한 태그와 조건 조합 체크
def is_valid_combination(tag, child_age):
    invalid_combos = {
        "태몽": ["1세", "2세"],
        "사주": ["0세"],
        "별자리 운세": ["0세"],
        "십이지 운세": ["0세"]
    }
    return child_age not in invalid_combos.get(tag, [])

In [3]:
# 데이터 프레임 초기화
data = []

# 각 태그별로 100개의 질문 생성
for tag in tags:
    for _ in tqdm(range(100), desc=f"Generating questions for tag: {tag}"):
        while True:
            # 각 비율에 따라 무작위 선택
            age = random.choices(ages, weights=age_ratio, k=1)[0]
            experience = random.choices(experiences, weights=experience_ratio, k=1)[0]
            gender = random.choices(genders, weights=gender_ratio, k=1)[0]
            child_age = random.choices(child_ages, weights=child_age_ratio, k=1)[0]
            season = random.choices(seasons, weights=season_ratio, k=1)[0]
            special_situation = random.choices(special_situations, weights=special_situation_ratio, k=1)[0]
            
            # 유효한 조합인지 확인
            if is_valid_combination(tag, child_age):
                break
        
        # 프롬프트 생성
        prompt = generate_prompt(tag, age, experience, gender, child_age, season, special_situation)
        
        # GPT-3.5-turbo API 호출
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100
        )
        
        # 생성된 질문 추출
        question = response['choices'][0]['message']['content'].strip()
        
        # 데이터 추가
        data.append({
            "tag": tag,
            "age": age,
            "experience": experience,
            "gender": gender,
            "child_age": child_age,
            "season": season,
            "special_situation": special_situation,
            "question": question
        })

# 데이터 프레임 생성
df = pd.DataFrame(data)

# CSV 파일로 저장
df.to_csv("generated_questions.csv", index=False)

print("Questions generation complete. Saved to 'generated_questions.csv'.")

Generating questions for tag: 태몽:   0%|          | 0/100 [00:00<?, ?it/s]

Generating questions for tag: 십이지 운세: 100%|██████████| 100/100 [02:27<00:00,  1.47s/it]

Questions generation complete. Saved to 'generated_questions.csv'.


: 